In [1]:
import pandas as pd
import numpy as np
# Read the CSV file and skip the first row (headers)


import pandas as pd

success_count = 0

for i in range(1, 100):
    filename = f"resources/statement ({i}).csv"
    
    try:
        bmo_statement = pd.read_csv(filename, skiprows=1)
        # Your code here using the bmo_statement
        
        success_count += 1
        
    except Exception as e:
        pass

print(f"{success_count} statements read successfully!")


exclude_strings = ['GOOGLE*YOUTUBEPREMIUM G.CO/HELPPAY#NS','TRSF FROM/DE ACCT/CPT','GOOGLE *YouTubePremium 650-253-0000 NS','FREEDOM MOBILE 877-946-3184 ON','ERIN MILLS EYE EXAM CL','CITY CENTRE BAPTIST','PROGRESSIVE REHABILITA','TRSF FROM/DE ACCT/CPT 3978-XXXX-729', 'RBC LIFE INSURANCE CO. MISSISSAUGA ON','AVIVA GENERAL INSURANC MARKHAM ON', 'ANNUAL CARD FEE', 'ANNUAL CARD FEE REBATE']
# Exclude rows containing any of the exclude_strings in the 'Description' column
bmo_statement = bmo_statement[~bmo_statement['Description'].str.contains('|'.join(exclude_strings))]
columns_to_remove = ['Item #', 'Card #', 'Posting Date']
bmo_statement = bmo_statement.drop(columns=columns_to_remove)
# Assuming you already have a dataframe called 'df' with the original column names

bmo_statement = bmo_statement.rename(columns={"Transaction Date": "Date",
                        "Transaction Amount": "Amount Debited",
                        "Description": "Source / Destination"})

bmo_statement["Date"] = pd.to_datetime(bmo_statement["Date"], format='%Y%m%d').dt.strftime('%Y-%m-%d')

bmo_statement = bmo_statement.reindex(columns=["Type", "Amount Debited", "Source / Destination", "Date"])
bmo_statement["Source / Destination"] = bmo_statement["Source / Destination"].str.replace(r"\bMISSISSAUGA ON\b|\bTORONTO ON\b", "", regex=True)

df1 = pd.read_csv('resources/purchase_type_updated.csv')
bmo_statement = bmo_statement.merge(df1, on='Source / Destination', how='left')
bmo_statement = bmo_statement.drop(['Unnamed: 0','Type_x'], axis=1)

# Assuming 'df' is the DataFrame where you want to rename the column
bmo_statement = bmo_statement.rename(columns={'Type_y': 'Type'})

# Check for NaN values
if bmo_statement.isnull().values.any():
    print("There are NaN values in the DataFrame.")

    # Find rows with missing values in the "Type" column
    mask = bmo_statement['Type'].isnull()
    missing_rows = bmo_statement[mask]

    # Loop through the missing rows and fill them in with user input
    for index, row in missing_rows.iterrows():
        source_dest = row['Source / Destination']
        value2 = input(f"Enter a value to fill NaNs for '{source_dest}': ")
        
        # Replace the missing value with the user input
        bmo_statement.loc[bmo_statement['Source / Destination'] == source_dest, 'Type'] = bmo_statement.loc[bmo_statement['Source / Destination'] == source_dest, 'Type'].replace(np.nan, value2)
        
        # Print out the subset of the DataFrame where 'Source / Destination' is equal to 'source_dest'
        subset_df = bmo_statement[bmo_statement['Source / Destination'] == source_dest]
        

else:
    print("There are no NaN values in the DataFrame.")

combined_summary = pd.read_csv('resources/combined_summary.csv')
combined_summary = pd.concat([combined_summary, bmo_statement])
combined_summary = combined_summary.filter(regex='^(?!Unnamed).*$')
combined_summary = combined_summary.drop_duplicates()
combined_summary = combined_summary.set_index('Type')
combined_summary.to_csv('resources/combined_summary.csv')
combined_summary


1 statements read successfully!
There are NaN values in the DataFrame.


,Amount Debited,Source / Destination,Date
Type,,,
Food,18.78,METRO 854,2022-02-23
Food,4.05,DERRY CONVENIENCE,2022-02-23
Food,101.42,RCSS 1080,2022-02-24
Food,34.92,METRO 854,2022-02-26
Food,44.36,DANFORTH FOOD MARKET,2022-03-12
...,...,...,...
Misc,903.35,HOLIDAY INN NIAGARA ON THON,2024-07-13
Misc,8.95,BULK BARN 545 MISSISSA,2024-07-13
Food,13.97,METRO 854,2024-07-14


In [2]:
df = pd.read_csv('resources/combined_summary.csv')
# Assuming 'df' is the existing DataFrame that you want to append 'bmo_statement' to
df = pd.concat([df, bmo_statement])
purchase_type_updated = df.drop(['Date','Amount Debited'], axis=1)
purchase_type_updated = purchase_type_updated.filter(regex='^(?!Unnamed).*$')
purchase_type_updated = purchase_type_updated.drop_duplicates()
purchase_type_updated.to_csv('resources/purchase_type_updated.csv')
purchase_type_updated

,Type,Source / Destination
0,Food,METRO 854
1,Food,DERRY CONVENIENCE
2,Food,RCSS 1080
4,Food,DANFORTH FOOD MARKET
5,Misc,CLORE BEAUTY SUPPLY
...,...,...
1722,Misc,HONK PARKING 866-675-7337 ON
1726,Misc,TRSF FROM/DE ACCT/CPT 0000022188313259
1728,Food,DOWNHILL HOT DOGS NIAGARA FALLSON
1729,Food,FUNNEL CAKES NIAGARA FALLSON
